In [1]:
import torch 
import torchvision
import numpy 
import torch.nn as nn
import torch.optim as optim
import tqdm

In [2]:
from torch.utils.data import DataLoader,Dataset
from torchvision.models import resnet50
import torchvision.transforms as transforms

In [3]:
train_data = torchvision.datasets.CIFAR10(root='/DATA/ravi10/Data/CIFAR_10',train=True,transform=transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
]),download=True)

test_data = torchvision.datasets.CIFAR10(root='/DATA/ravi10/Data/CIFAR_10',train=False,transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
]),download=True)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_loader = DataLoader(train_data,batch_size=32,shuffle=True,num_workers=4)
test_loader = DataLoader(test_data,batch_size=32,shuffle=True,num_workers=4)

In [5]:
base = resnet50(pretrained=True)

In [6]:
base

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
class Identity(nn.Module):
  def __init__(self):
    super(Identity,self).__init__()

  def forward(self,x):
    return x


# Feature Extraction

In [8]:
base.fc = Identity()

In [9]:
for param in base.parameters():
    param.requires_grad = False

In [10]:
model = nn.Sequential(
    base,
    torch.nn.Flatten(),
    nn.Linear(2048,1024),
    nn.ReLU(),
    nn.Linear(1024,10),
    nn.Sigmoid()
)

In [11]:
model

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [12]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
#count_parameters(model)

In [13]:
optimizer = optim.Adam(model.parameters(),lr=0.0001)
get_loss = nn.CrossEntropyLoss(reduction='mean')

In [14]:
device='cuda:4'
# if torch.cuda.device_count() > 1:
#     model = nn.DataParallel(model, device_ids=[4,6,7])
model.to(device)
#from tqdm import tqdm
import tqdm

In [15]:
def train():
  model.train()
  for i,(input,target) in tqdm.tqdm(enumerate(train_loader)):
    optimizer.zero_grad()
    target = target.to(device)
    input = input.to(device)
    preds = model(input)
    loss = get_loss(preds,target)
    loss.backward()
    optimizer.step()
  print(loss.data)


In [16]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
      for input,target in test_loader:
        target = target.to(device)
        input = input.to(device)
        preds = model(input)
        test_loss += get_loss(preds,target)
        label = torch.max(preds,1)[1]
        correct += label.eq(target.view_as(label)).cpu().sum()
      test_loss /= len(test_loader.dataset)
      acc = 100.*correct/len(test_loader.dataset)
      print('\nTest Error:{:.4f} Correct:{} Accuracy:{:.4f}'.format(test_loss,correct,acc))


In [17]:
for i in range(10):
  train()
  test()

1563it [01:38, 15.90it/s]


tensor(1.6684, device='cuda:4')

Test Error:0.0529 Correct:7542 Accuracy:75.4200


1563it [01:37, 16.07it/s]

tensor(1.7195, device='cuda:4')



Test Error:0.0520 Correct:7685 Accuracy:76.8500


...

tensor(1.6104, device='cuda:4')



Test Error:0.0507 Correct:7944 Accuracy:79.4400


1563it [01:36, 16.14it/s]

tensor(1.6137, device='cuda:4')



Test Error:0.0507 Correct:7953 Accuracy:79.5300


# Fine tuning

In [18]:
import re
for name, parameter in base.named_parameters():
  #print(name)
  s = str(name)
  if (s.count('layer4.2')>0):
    #print(name)
    parameter.requires_grad = True
  else:
    #print("hello")
    parameter.requires_grad = False

In [19]:
count_parameters(base)

+-----------------------+------------+
|        Modules        | Parameters |
+-----------------------+------------+
| layer4.2.conv1.weight |  1048576   |
|  layer4.2.bn1.weight  |    512     |
|   layer4.2.bn1.bias   |    512     |
| layer4.2.conv2.weight |  2359296   |
|  layer4.2.bn2.weight  |    512     |
|   layer4.2.bn2.bias   |    512     |
| layer4.2.conv3.weight |  1048576   |
|  layer4.2.bn3.weight  |    2048    |
|   layer4.2.bn3.bias   |    2048    |
+-----------------------+------------+
Total Trainable Params: 4462592


4462592

In [20]:
model = nn.Sequential(
    base,
    torch.nn.Flatten(),
    nn.Linear(2048,1024),
    nn.ReLU(),
    nn.Linear(1024,10),
    nn.Sigmoid()
)
device = torch.device('cuda:3')
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model, device_ids=[3,4,5])
model.to(device)

DataParallel(
  (module): Sequential(
    (0): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace

In [21]:

count_parameters(model)

+--------------------------------+------------+
|            Modules             | Parameters |
+--------------------------------+------------+
| module.0.layer4.2.conv1.weight |  1048576   |
|  module.0.layer4.2.bn1.weight  |    512     |
|   module.0.layer4.2.bn1.bias   |    512     |
| module.0.layer4.2.conv2.weight |  2359296   |
|  module.0.layer4.2.bn2.weight  |    512     |
|   module.0.layer4.2.bn2.bias   |    512     |
| module.0.layer4.2.conv3.weight |  1048576   |
|  module.0.layer4.2.bn3.weight  |    2048    |
|   module.0.layer4.2.bn3.bias   |    2048    |
|        module.2.weight         |  2097152   |
|         module.2.bias          |    1024    |
|        module.4.weight         |   10240    |
|         module.4.bias          |     10     |
+--------------------------------+------------+
Total Trainable Params: 6571018


6571018

In [22]:

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [23]:
for i in range(20):
    print("epoch:",i)
    train()
    scheduler.step()
    if i%10 == 0:
        test()
  #test()

epoch: 0


1563it [02:27, 10.57it/s]

tensor(1.9403, device='cuda:3')



Test Error:0.0562 Correct:8544 Accuracy:85.4400
epoch: 1


1563it [02:23, 10.91it/s]

...


Test Error:0.0484 Correct:9137 Accuracy:91.3700
epoch: 21


1563it [02:21, 11.04it/s]

tensor(1.6339, device='cuda:3')
epoch: 22



1448it [02:12, 10.96it/s]


KeyboardInterrupt: 

In [24]:
test()


Test Error:0.0483 Correct:9156 Accuracy:91.5600


In [ ]:
torch.save(model.state_dict(),'./resnet.h5')

In [ ]:
#model.load_state_dict(torch.load('./resnet.h5'))